In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import copy

# Setup options
options = Options()
# options.add_argument("--headless")  # Uncomment after testing
options.add_argument("--window-size=1920,1080")

# Start the browser
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 10)

driver.get(" https://www.iq-zert.de/personen_wertermittlung_akkr.html")
time.sleep(2)

response=[]

In [15]:
def extract_results():
    try:
        expert_rows = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, "//tbody/tr"))
        )

        for row in expert_rows:
            try:
                cols = row.find_elements(By.TAG_NAME, "td")
                if len(cols) < 10:
                    continue  # skip incomplete or empty rows

                # Extract fields
                last_name = cols[2].text.strip()
                first_name = cols[3].text.strip()
                full_name = f"{first_name} {last_name}".strip()

                postcode = cols[5].text.strip()
                location = cols[6].text.strip()
                address = f"{postcode} {location}".strip()

                phone = cols[7].text.strip()
                email = cols[8].text.strip()
                website = cols[9].text.strip()

                # Append to response
                response.append({
                    "Full Name": full_name,
                    "Address": address,
                    "Phone Number": phone,
                    "Email Address": email,
                    "Website": website,
                    "Certification Type / Details": "",  # Not available in this table
                    "Source Directory Name": "IQ-ZERT"
                })

            except Exception as block_err:
                print(f"❗ Error extracting a row: {str(block_err)}")

    except TimeoutException:
        print("⏱️ Timeout: No rows found; check table selector")
    except Exception as e:
        print(f"❗ Unexpected error: {str(e)}")


In [16]:
while True:
    extract_results()
    try:
        next_button = driver.find_element(By.XPATH, '//a[@class="paginate_button next"]')
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(3)
    except Exception as e:
        print(f"🔁 All the Data Extract from the website")
        break

🔁 All the Data Extract from the website


In [17]:
print(response)

[{'Full Name': 'Ahmad Barjawi', 'Address': '69207 Sandhausen', 'Phone Number': '0157 58508176', 'Email Address': 'kontakt@wertweiser-immobilienbewertung.de', 'Website': 'www.wertweiser-immobilienbewertung.de', 'Certification Type / Details': '', 'Source Directory Name': 'IQ-ZERT'}, {'Full Name': 'Ahmadi Daryoush Torkaman', 'Address': '44809 Bochum', 'Phone Number': '0172 5665041', 'Email Address': 'daryo_18@hotmail.de', 'Website': '', 'Certification Type / Details': '', 'Source Directory Name': 'IQ-ZERT'}, {'Full Name': 'Alexander Mayrhofer', 'Address': '41061 Mönchengladbach', 'Phone Number': '02161 831496', 'Email Address': 'info@alexander-mayrhofer.de', 'Website': '', 'Certification Type / Details': '', 'Source Directory Name': 'IQ-ZERT'}, {'Full Name': 'Amadeus Bender', 'Address': '34125 Kassel', 'Phone Number': '01573 7545210', 'Email Address': 'amadeusbender@googlemail.com', 'Website': '', 'Certification Type / Details': '', 'Source Directory Name': 'IQ-ZERT'}, {'Full Name': 'And

In [18]:
backup_response = copy.deepcopy(response)
print(f"🔎 Original: {len(backup_response)} entries")

🔎 Original: 81 entries


In [19]:
# Clean: remove entries without an address
response = [entry for entry in response if entry.get("Address")]
print(f"✅ After cleaning: {len(response)} entries")

✅ After cleaning: 81 entries


In [21]:
df = pd.DataFrame(response)
df.drop_duplicates(inplace=True)
df.to_csv("IQ-ZERT_results.csv", index=False)
df.to_excel("IQ-ZERT_results.xlsx", index=False)